In [ ]:
 %matplotlib inline

import os
from dateutil.relativedelta import relativedelta

import numpy as np
import pandas as pd
from bson.objectid import ObjectId
import matplotlib.pyplot as plt
from mongoengine import connect
from pycoshark.mongomodels import Project, VCSSystem, Commit, Tag, File, CodeEntityState, FileAction, People, IssueSystem, Issue, Message, MailingList, Event, MynbouData, Identity, Hunk, Branch
from pycoshark.utils import java_filename_filter, jira_is_resolved_and_fixed, get_commit_graph, heuristic_renames

loc = {'host': '127.0.0.1',
       'port': 27018,
       'db': 'smartshark',
       'username': 'root',
       'password': 'balla',
       'authentication_source': 'smartshark',
       'connect': False}
connect(**bimbes)

In [ ]:
dfall = pd.read_pickle('/srv/www/emse2019_replication/data/aggregated2_full.pickle')
len(dfall)

In [ ]:
code_files = set()
test_files = set()
for project in dfall['project'].unique():
    p = Project.objects.get(name=project)
    vcs = VCSSystem.objects.get(project_id=p.id)

    # sample 1% of the commits used in the study
    commit_sample = dfall[dfall['project'] == project].sample(frac=0.01, replace=False, random_state=42)['revision'].values

    # get changed files from the database and write them to the samples file for manual inspection
    for commit in commit_sample:
        c = Commit.objects.only('id').get(vcs_system_id=vcs.id, revision_hash=commit)
        for fa in FileAction.objects.filter(commit_id=c.id):
            f = File.objects.get(id=fa.file_id)
            if not f.path.lower().endswith('.java'):
                continue
            if java_filename_filter(f.path, production_only=True):
                code_files.add(f.path)
            else:
                test_files.add(f.path)

# collect samples for each project
with open('../file_samples.txt', 'w') as fd:
    fd.write('code files:')
    fd.write('\n'.join(code_files))
    fd.write('\n')
    fd.write('test files:')
    fd.write('\n'.join(test_files))